# データ分析

## 目的


## データ概要



In [1]:
# ライブラリのインポート
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 設定
plt.style.use('default')
sns.set_palette('husl')
%matplotlib inline

# 表示設定
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [4]:
from google.cloud import storage
from google.oauth2 import service_account
from collections import defaultdict
import os
import concurrent.futures

# ==========================================
# 設定項目
# ==========================================
BUCKET_NAME = "medicmedia_user_dashboard" # ★バケット名を忘れずに！
ROOT_PREFIX = "qa_view_rate/"

# ★JSONキーを使う場合（uv等は環境変数推奨）
SERVICE_ACCOUNT_FILE = os.path.join('..', 'gcp-qb3online-key.json')

DRY_RUN = False   # ★まずはTrueで確認！
MAX_WORKERS = 50 # タスクが細かくなるので並列数を少し増やしました
# ==========================================

def get_client():
    """認証済みクライアントを取得"""
    if 'SERVICE_ACCOUNT_FILE' in globals():
        credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE)
        return storage.Client(credentials=credentials, project=credentials.project_id)
    return storage.Client()

def get_subdirectories(bucket, prefix):
    """指定パス直下のディレクトリ一覧を取得する（delimiter='/'を利用）"""
    iterator = bucket.list_blobs(prefix=prefix, delimiter='/')
    list(iterator) # データ取得を実行
    return list(iterator.prefixes)

def find_target_directories(bucket, root_prefix):
    """
    ルートから college_grade 階層まで掘り下げて、処理対象フォルダのリストを作成する
    構造: root -> is_exam -> job_category_id -> college_grade (ここまで掘る)
    """
    print("フォルダ構造を解析中...")
    
    # 1. is_exam レベルを取得
    exam_dirs = get_subdirectories(bucket, root_prefix)
    if not exam_dirs:
        print("  Warning: is_exam ディレクトリが見つかりません。")
        return [root_prefix] # 仕方がないのでルートを返す

    target_dirs = []

    # 2. job_category_id レベルを取得
    for exam_dir in exam_dirs:
        job_dirs = get_subdirectories(bucket, exam_dir)
        
        for job_dir in job_dirs:
            # 3. college_grade レベルを取得（ここを並列処理の単位にする）
            grade_dirs = get_subdirectories(bucket, job_dir)
            
            if grade_dirs:
                target_dirs.extend(grade_dirs)
            else:
                # college_gradeがない場合（ファイル直置きなど）、このjobフォルダを対象にする
                target_dirs.append(job_dir)

    print(f"  解析完了: 並列処理対象のディレクトリ数（college_grade単位）: {len(target_dirs)}")
    return target_dirs

def process_directory_recursive(bucket_name, prefix):
    """
    割り当てられたディレクトリ（college_grade）以下の全JSONを整理する
    """
    client = get_client()
    bucket = client.bucket(bucket_name)
    
    # 再帰的に全ファイルを取得
    blobs = list(bucket.list_blobs(prefix=prefix))
    
    files_by_dir = defaultdict(list)
    for blob in blobs:
        if blob.name.endswith('.json'):
            parent = os.path.dirname(blob.name)
            files_by_dir[parent].append(blob)

    deleted_count = 0
    processed_dirs = 0
    
    for dir_path, blob_list in files_by_dir.items():
        if len(blob_list) <= 1:
            continue
        
        processed_dirs += 1
        # 新しい順にソート
        blob_list.sort(key=lambda x: x.time_created, reverse=True)
        
        # 2つ目以降を削除
        duplicates = blob_list[1:]
        for blob in duplicates:
            if not DRY_RUN:
                try:
                    blob.delete()
                except Exception:
                    pass # エラーは無視して続行
            deleted_count += 1
            
    return processed_dirs, deleted_count

def clean_gcs_deep_parallel(bucket_name, root_prefix):
    client = get_client()
    bucket = client.bucket(bucket_name)

    print(f"処理開始: gs://{bucket_name}/{root_prefix}")

    # 1. 処理対象のディレクトリ（college_gradeレベル）をリストアップ
    target_prefixes = find_target_directories(bucket, root_prefix)

    if not target_prefixes:
        print("処理対象が見つかりませんでした。パスを確認してください。")
        return

    # 2. 並列実行
    total_deleted = 0
    total_dirs_checked = 0

    print(f"\n並列処理を開始します（Workers: {MAX_WORKERS}）...")
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        # タスク割り当て
        future_to_prefix = {
            executor.submit(process_directory_recursive, bucket_name, p): p 
            for p in target_prefixes
        }

        # 進捗表示用
        finished_count = 0
        total_tasks = len(target_prefixes)

        for future in concurrent.futures.as_completed(future_to_prefix):
            prefix = future_to_prefix[future]
            finished_count += 1
            try:
                p_dirs, deleted = future.result()
                total_dirs_checked += p_dirs
                total_deleted += deleted
                
                # 進捗ログ（少し間引いて表示）
                if finished_count % 10 == 0 or finished_count == total_tasks:
                    print(f"進捗: {finished_count}/{total_tasks} 完了 (削除数累計: {total_deleted})")
                    
            except Exception as e:
                print(f"[Error] {prefix}: {e}")

    print("-" * 40)
    print("全処理完了")
    if DRY_RUN:
        print(f"[DRY RUN結果] 整理対象ディレクトリ: {total_dirs_checked}, 削除予定ファイル: {total_deleted}")
        print("※ 実際に削除するには DRY_RUN = False にしてください。")
    else:
        print(f"[実行結果] 整理したディレクトリ: {total_dirs_checked}, 削除したファイル: {total_deleted}")

if __name__ == "__main__":
    clean_gcs_deep_parallel(BUCKET_NAME, ROOT_PREFIX)

処理開始: gs://medicmedia_user_dashboard/qa_view_rate/
フォルダ構造を解析中...
  解析完了: 並列処理対象のディレクトリ数（college_grade単位）: 895

並列処理を開始します（Workers: 50）...
進捗: 10/895 完了 (削除数累計: 10)
進捗: 20/895 完了 (削除数累計: 20)
進捗: 30/895 完了 (削除数累計: 30)
進捗: 40/895 完了 (削除数累計: 40)
進捗: 50/895 完了 (削除数累計: 50)
進捗: 60/895 完了 (削除数累計: 60)
進捗: 70/895 完了 (削除数累計: 70)
進捗: 80/895 完了 (削除数累計: 80)
進捗: 90/895 完了 (削除数累計: 90)
進捗: 100/895 完了 (削除数累計: 100)
進捗: 110/895 完了 (削除数累計: 110)
進捗: 120/895 完了 (削除数累計: 120)
進捗: 130/895 完了 (削除数累計: 130)
進捗: 140/895 完了 (削除数累計: 140)
進捗: 150/895 完了 (削除数累計: 150)
進捗: 160/895 完了 (削除数累計: 160)
進捗: 170/895 完了 (削除数累計: 170)
進捗: 180/895 完了 (削除数累計: 180)
進捗: 190/895 完了 (削除数累計: 190)
進捗: 200/895 完了 (削除数累計: 200)
進捗: 210/895 完了 (削除数累計: 209)
進捗: 220/895 完了 (削除数累計: 219)
進捗: 230/895 完了 (削除数累計: 229)
進捗: 240/895 完了 (削除数累計: 239)
進捗: 250/895 完了 (削除数累計: 249)
進捗: 260/895 完了 (削除数累計: 259)
進捗: 270/895 完了 (削除数累計: 269)
進捗: 280/895 完了 (削除数累計: 279)
進捗: 290/895 完了 (削除数累計: 289)
進捗: 300/895 完了 (削除数累計: 299)
進捗: 310/895 完了 (削除数累計: 309)
進捗: 320/895 